### Pomocné třídní dekorátory
---

<br>

#### Metody v OOP

---

Jako ucelený seznam metod, které můžeme použít v rámci OOP se podívej na tyto metody:
1. **Instanční** metoda,
2. **třídní** metoda,
3. **statická** metoda.

<br>

Celou situaci lze vysvětlit na teoretické ukázce:

In [3]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def load_data(self):  # klasická metoda instance
        return "Calling instance method..", self
    
    @classmethod
    def parsing_files(cls):  # třídní metoda
        return "Calling class method..", cls
    
    @staticmethod
    def is_there_file():     # statická metoda
        return "Calling static method.."

In [4]:
reader = KindleNotesParser()

<br>

##### Instační metoda
Metoda, která má mj. mezi parametry klíčový výraz `self`. Tím dovede zpřístupnit jak třídy (a její atributy), tak instance.

In [5]:
print(
    reader,              # adresa objektu instance 'reader'
    reader.load_data,    # popis metody patřící instanci 'reader'
    reader.load_data(),  # spuštění, opět adresa
    sep="\n"
)

<bound method KindleNotesParser.load_data of <__main__.KindleNotesParser object at 0x7ff0f648cf70>>
('Calling instance method..', <__main__.KindleNotesParser object at 0x7ff0f648cf70>)


<br>

Takže pomocí předchozí ukázky můžeme říct, že **instanční metoda** je schopna zpřístupnit *původní třídu* a případně pracovat s jejími *atributy*.

<br>

##### Třídní metoda
Podobný zápis jako pro instační metodu, ale liší se dvěma zásadním rozdíly:
1. Dekorátor `@classmethod`,
2. parametr `cls`.

In [6]:
print(
    reader.parsing_files,    # adresa objektu instance nyní chybí
    reader.parsing_files(),  # metodu spouští, třídy zpřístupní, ale instanci nezná
    sep="\n"
)

<bound method KindleNotesParser.parsing_files of <class '__main__.KindleNotesParser'>>
('Calling class method..', <class '__main__.KindleNotesParser'>)


<br>

Jakmile použiješ **třídní metodu**, vidíš že máš přístup k objektu *původní třídy*, ale tentokrát není k dispozici odkaz (adresa) *instance*.

```
('Reading .txt file..', <__main__.KindleParser object at 0x7f0ed30782e0>)
('Spoustim metodu tridy', <class '__main__.KindleParser'>)
```

<br>

##### Statická metoda

1. Dekorátor `@staticmethod`,
2. chybí parametr `self`,
3. chybí parametr `cls`.

In [7]:
print(
    reader.is_there_file,
    reader.is_there_file(),
    sep="\n"
)

<function KindleNotesParser.is_there_file at 0x7ff0f43929d0>
Calling static method..


In [8]:
print(KindleNotesParser.load_data())

TypeError: load_data() missing 1 required positional argument: 'self'

In [9]:
print(KindleNotesParser.parsing_files())

('Calling class method..', <class '__main__.KindleNotesParser'>)


In [10]:
print(KindleNotesParser.is_there_file())

Calling static method..


<br>

Pokud spustíš a prozkoumáš **statickou metodu**, můžeš si ověřit, že tato metoda nemá přístup ani k *původní třídě*, ani k její *instanci*.

<br>

##### Shrnutí k metodám

---
1. **instanční metoda** - může upravit nejenom objekty instance, ale i třídy (na začátku vidí jak třídu, tak instanci),
2. **třídní metoda (@classmethod)** - může upravit objekty třídy, ale nemůže upravovat objekty instancí (vidí třídu, ale ne instanci),
3. **statická metoda (@staticmethod)** - nemůže upravovat ani objekty instancí, ani objekty třídy (nevidí ani třídu, ani instanci).

<br>

##### Praktické ukázky

---

###### Instanční metoda

In [11]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    notes: list = []  # upravení třídního atributu
    
    def __init__(self, file: str, data: str):
        self.file = file
        self.data = data
#         self.notes = list()  # upravení instan. atributu
        
    def load_data(self):
        return self.notes.append(self.data)

In [12]:
first_note = KindleNotesParser("poznamky.txt", "Moje první poznámka k ...")
second_note = KindleNotesParser("poznamky_nove.txt", "Druhá poznámka ke knížce ...")

first_note.load_data()
second_note.load_data()

print(first_note.notes, second_note.notes, sep="\n")

['Moje první poznámka k ...', 'Druhá poznámka ke knížce ...']
['Moje první poznámka k ...', 'Druhá poznámka ke knížce ...']


<br>

Je jedno, který atribut budeš chtít upravit. Díky **instanční metodě** můžeš pracovat jak s třídními, tak s instančními objekty.
<br>

###### Třídní metoda

In [13]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    readed_files: int = 0
    
    def __init__(self, file: str):
        self.file = file
    
    @classmethod
    def parsing_files(cls, name):
        instance = cls(name)
        cls.readed_files += 1  
        print(f"Parsing data from: {name}")

In [14]:
print(f"{KindleNotesParser.readed_files=}")

KindleNotesParser.readed_files=0


In [15]:
KindleNotesParser.parsing_files("poznamky.txt")
KindleNotesParser.parsing_files("nove_poznamky.txt")
KindleNotesParser.parsing_files("poznpozn.txt")

Parsing data from: poznamky.txt
Parsing data from: nove_poznamky.txt
Parsing data from: poznpozn.txt


In [16]:
print(f"{KindleNotesParser.readed_files=}")

KindleNotesParser.readed_files=3


<br>

Pokud budeš chtít použít **třídní metody**, potom dávej pozor na to, že můžeš spravovat pouze třídní atributy.
<br>

###### Statická metoda

In [17]:
import os

class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def __init__(self, file: str):
        self.file = file
    
    @staticmethod
    def is_there_file(name):
        print("The file exists!") if os.path.exists(name) else print("Does not exist!")


In [18]:
parser = KindleNotesParser("")
parser.is_there_file("lesson01.ipynb")

The file exists!


In [19]:
parser.is_there_file("lesson11.ipynb")

Does not exist!


<br>

**Statická metoda** nepotřebuje vědět nic ani o třídě, ani o instanci. Pracuje s parametrem jako klasická funkce. Ale svým účelem spadá jako nástroj ke konkrétní třídě.

<br>

#### Vlastnosti třídy

---

Představ si situaci, kdy máš napsat převodník jednotek objemu. Z **litrů** na **pinty**(UK):

In [50]:
class LiterConvertor:
    def __init__(self, liter: int = 0):
        self.liter = liter
        
    def to_pints(self):
        return self.liter * 1.759754

In [51]:
bottle_volume = LiterConvertor()
bottle_volume.liter = 0.75
print(bottle_volume.liter)
print(bottle_volume.to_pints())

0.75
1.3198155


<br>

Kdykoliv se pokusíš tímto způsobem přepsat hodnotu, interpret pracuje stejně jako když přepisuješ hodnotu ve slovníku:

In [55]:
 print(bottle_volume.__dict__)

{'liter': 0.75}


In [54]:
print(
    bottle_volume.liter,              # instační atribut
    bottle_volume.__dict__["liter"],  # slovníkový výběr podle klíče
    sep="\n"
)

0.75
0.75


<br>

Lepší implementace (pythonovější) v rámci objektově-orientovaného programování by vypadala následovně:

In [39]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.set_volume_in_liter(liter)
        
    def to_pints(self):
        return self.get_volume_in_liter() * 1.759754
    
    def get_volume_in_liter(self):
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value

In [40]:
volume = LiterConvertor(0.75)
print(
    volume.get_volume_in_liter(),
    volume.to_pints(),
    sep="\n"
)

0.75
1.3198155


In [41]:
volume.set_volume_in_liter(-1)

ValueError: Cannot process negative number

<br>

Toto řešení nám dovolilo přidat omezení, kdy nechceme počítat objem s negativní hodnotou.

<br>

Bohužel se také objevil problém s ohledem na implementaci našeho vylepšení. Všechny ohlášení `obj.liter` je nutné přepsat na `obj.get_volume_in_liter()` a `obj.liter = val` na `obj.set_volume(val)`.

<br>

Taková úprava řešení může znamenat problém na desítky, stovky řádků.

<br>

Naštěstí můžeme s vlastnostmi třídy nakládat lépe!

In [48]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.liter = liter
        
    def to_pints(self):
        return self.liter * 1.759754
    
    def get_volume_in_liter(self):
        print("Getting..")
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        print("Setting..")
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value
        
    liter = property(get_volume_in_liter, set_volume_in_liter)

In [49]:
volume = LiterConvertor(0.75)
print(volume.liter)
print(volume.to_pints())

Setting..
Getting..
0.75
Getting..
1.3198155


In [47]:
volume.liter = -1

Setting..


ValueError: Cannot process negative number

<br>

Pokaždé co tentokrát použijeme proměnnou `liter` dojde automaticky k zavolání metody `get_volume_in_liter()`.

<br>

Stejně tak, pokud budeš chtít přepast hodnotu v proměnnou `liter`, tak dojde ke spuštění metody `set_volume_in_liter()`.

<br>

Takže pokud použiješ funkci `property()`, nemusíš se omezovat na potřebné úpravy tvého stávajícího skriptu.

<br>


#### Dekorátory setter, getter, deleter

---

<br>